In [2]:
from pyspark import SparkConf, SparkContext

1. Creamos un contexto nuevo de Spark para leer un archivo de texto

In [3]:
conf = SparkConf().setAppName('readFile')
spark = SparkContext.getOrCreate(conf=conf)

your 131072x1 screen size is bogus. expect trouble


24/02/27 20:50:23 WARN Utils: Your hostname, DESKTOP-SLEQT56 resolves to a loopback address: 127.0.1.1; using 172.25.13.138 instead (on interface eth0)
24/02/27 20:50:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/02/27 20:50:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
text = spark.textFile("/mnt/d/Proyectos/Tutorial-SparkAWS/data/sampletext.txt")

In [5]:
type(text)

pyspark.rdd.RDD

In [6]:
text.collect()

['1 2 3 4', '10 23 45 67', '45 65 34 56', '87 54 34 101']

2. Sobre el __rdd__ aplicamos una __transformacion__ map que devuelve otro __rdd__

In [7]:
miFile_rdd = text.map(lambda x:x.split(' '))


In [8]:
type(miFile_rdd)

pyspark.rdd.PipelinedRDD

In [9]:
miFile_rdd.collect()

[['1', '2', '3', '4'],
 ['10', '23', '45', '67'],
 ['45', '65', '34', '56'],
 ['87', '54', '34', '101']]

3. En este ejemplo queremos usar una funcion __foo__ definida por el usuario donde, primero hagamos lo mismo que se hizo en __2__ pero luego se castea cada elemento a __int__ y se le suma 2.

In [10]:
def mi_foo(x):
    l =  x.split(' ')
    ls = []
    for num in l:
        ls.append(int(num) + 2)
    return ls

miFile  = text.map(mi_foo)
miFile.collect()

[[3, 4, 5, 6], [12, 25, 47, 69], [47, 67, 36, 58], [89, 56, 36, 103]]

Ejercicio.

Tenemos un archivo con saludos (texto) y debemos devolver un array con la logitud de cada palabra.

In [12]:
fileText = spark.textFile("/mnt/d/Proyectos/Tutorial-SparkAWS/data/wordcount.txt")

In [14]:
fileText.collect()

['Hi how are you?', 'Hope you are doing', 'great']

In [15]:
def foo(x):
    array = x.split(' ')
    l2 = []
    for word in array:
        l2.append(len(word))
    return l2

fileText.map(foo).collect()

[[2, 3, 3, 4], [4, 3, 3, 5], [5]]

Variacion del ejercicio.

En lugar de usar una __UDF__ usamos una funcion __lambda__ con una __list comprehention__ para obtener el mismo resultado.

In [29]:
fileText.map(lambda x: [len(word) for word in x.split(' ')]).collect()

[[2, 3, 3, 4], [4, 3, 3, 5], [5]]

4. FlatMap

Devuelve todos los elementos en un tipo de dato __unificado__  generando un nuevo __rdd__

In [36]:
fileText.flatMap(lambda x:x.split(' ')).collect()

['Hi', 'how', 'are', 'you?', 'Hope', 'you', 'are', 'doing', 'great']

5. filter - lambda

Se usa para segmentar el resultado de una __rdd__

In [39]:
text.filter(lambda x: x in ('10 23 45 67', '87 54 34 101')).collect()

['10 23 45 67', '87 54 34 101']

Usar filter sin __lambda function__

In [52]:
def foo(x):
    if x in ('10 23 45 67', '87 54 34 101'):
        return True
    return False


text.filter(foo).collect()

['10 23 45 67', '87 54 34 101']